![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [5]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV



filename_features = "C:/DEV/Datasets/EBAC/PCA/UCI HAR Dataset/features.txt"
filename_labels = "C:/DEV/Datasets/EBAC/PCA/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "C:/DEV/Datasets/EBAC/PCA/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "C:/DEV/Datasets/EBAC/PCA/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "C:/DEV/Datasets/EBAC/PCA/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "C:/DEV/Datasets/EBAC/PCA/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "C:/DEV/Datasets/EBAC/PCA/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "C:/DEV/Datasets/EBAC/PCA/UCI HAR Dataset/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], squeeze=True, sep="#")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'], squeeze=True)
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'], squeeze=True)
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

C:\Users\Lucia\AppData\Local\Temp\ipykernel_28768\234977206.py:23: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  features = pd.read_csv(filename_features, header=None, names=['nome_var'], squeeze=True, sep="#")
C:\Users\Lucia\AppData\Local\Temp\ipykernel_28768\234977206.py:26: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'], squeeze=True)
C:\Users\Lucia\AppData\Local\Temp\ipykernel_28768\234977206.py:30: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'], squeeze=True)


## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [8]:
%%time

# Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

dt = DecisionTreeClassifier(ccp_alpha=0.001)
dt.fit(X_train, y_train)

# avaliando a acurácia na base de treinamento e teste
y_pred_train = dt.predict(X_train)
y_pred_test = dt.predict(X_test)

acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)

print(f'Acurácia na base de treinamento: {acc_train:.2f}')
print(f'Acurácia na base de teste: {acc_test:.2f}')

Acurácia na base de treinamento: 0.98
Acurácia na base de teste: 0.88
CPU times: total: 1.66 s
Wall time: 3.08 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [9]:
%%time

# utilizando pca para reduzir a dimensionalidade para 1 componente
pca = PCA(n_components=1).fit(X_train)

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

# Rode uma árvore de decisão com a variável reduzida, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

dt = DecisionTreeClassifier(ccp_alpha=0.001)
dt.fit(X_train_pca, y_train)

# avaliando a acurácia na base de treinamento e teste
y_pred_train = dt.predict(X_train_pca)
y_pred_test = dt.predict(X_test_pca)

acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)

print(f'Acurácia na base de treinamento: {acc_train:.2f}')
print(f'Acurácia na base de teste: {acc_test:.2f}')



Acurácia na base de treinamento: 0.50
Acurácia na base de teste: 0.46
CPU times: total: 109 ms
Wall time: 177 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [19]:
%%time
from sklearn.pipeline import Pipeline

# utilizar o GridSearchCV para treinar arvores de decisão com diferentes quantidades de componentes principais

pca = PCA()
dt = DecisionTreeClassifier(ccp_alpha=0.001)

pipe = Pipeline(steps=[('pca', pca), ('dt', dt)])

param_grid = {
    'pca__n_components': [1, 3, 5, 10, 20, 50],
}

search = GridSearchCV(pipe, param_grid, n_jobs=-1)

search.fit(X_train, y_train)

# mostrar todos os resultados do GridSearchCV incluindo o tempo de processamento de cada modelo
results = pd.DataFrame(search.cv_results_)
results.sort_values('rank_test_score', inplace=True)
print(results[['params', 'mean_test_score', 'mean_fit_time']], '\n')
print("Melhor acurácia:", search.best_score_)
print("Acurácia na base de teste:", search.score(X_test, y_test))



                      params  mean_test_score  mean_fit_time
3  {'pca__n_components': 10}         0.799102       0.526873
5  {'pca__n_components': 50}         0.795428       0.589750
4  {'pca__n_components': 20}         0.789035       0.564806
2   {'pca__n_components': 5}         0.774078       0.379033
1   {'pca__n_components': 3}         0.720754       0.340822
0   {'pca__n_components': 1}         0.462866       0.469593 

Melhor acurácia: 0.799101726346555
Acurácia na base de teste: 0.8215134034611469
CPU times: total: 453 ms
Wall time: 1.99 s


## Conclua

- O que aconteceu com a acurácia?  
      A acurácia decaiu conforme o número de componentes aumentou. não superando a acurácia do modelo com 10 componentes.
      
- O que aconteceu com o tempo de processamento?  
      O tempo de processamento aumentou conforme o número de componentes aumentou. Mas o tempo de processamento do próprio GridSearchCV foi 
    menor, menos da metade do tempo de processamento do modelo sem utilizar o PCA. Utilizando um número fixo de componentes, poderiamos treinar
    um dataset muitas vezes maior no mesmo tempo.